### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "../Resources/schools_complete.csv"
student_data_to_load = "../Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:
# Calculate the total number of schools
unique_school_name=school_data_complete['school_name'].unique()
total_school_number=len(unique_school_name)

# Calculate the total number of students
total_students_number=school_data_complete["student_name"].count()

# Calculate the total budget
total_budget = school_data["budget"].sum()

# Calculate the average math score.
average_math_score = school_data_complete["math_score"].mean()

# Calculate the average reading score
average_reading_score = school_data_complete["reading_score"].mean()

#Calculate the percentage of students with a passing math score (70 or greater)
students_passing_math = school_data_complete.loc[school_data_complete["math_score"] >= 70]
number_students_passing_math = students_passing_math["Student ID"].count()

percent_passing_math = (number_students_passing_math / total_students_number) * 100

# Calculate the percentage of students with a passing reading score (70 or greater)
students_passing_reading = school_data_complete.loc[school_data_complete["reading_score"] >= 70]
number_students_passing_reading = students_passing_reading["Student ID"].count()

percent_passing_reading = (number_students_passing_reading / total_students_number) * 100

# Calculate the percentage of students who passed math and reading (% Overall Passing)
Overall_passing = school_data_complete[(school_data_complete['math_score'] >= 70) & (school_data_complete['reading_score'] >= 70)]['Student ID'].count()/total_students_number*100

# Create a dataframe to hold the above results
district_summary = pd.DataFrame({
    "Total Schools": total_school_number,
    "Total Students": f"{total_students_number:,}",
    "Total Budget": f"${total_budget:,.2f}",
    "Average Math Score": f"{average_math_score:.6f}",
    "Average Reading Score": f"{average_reading_score:.5f}",
    "% Passing Math": f"{percent_passing_math:.6f}",
    "% Passing Reading": f"{percent_passing_reading:.6f}",
    "% Overall Passing": f"{Overall_passing: .6f}"
}, index=[0])

district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [3]:
# Group by school name
school_name = school_data_complete.set_index('school_name').groupby(['school_name'])

# school types by school name
school_type = school_data.set_index('school_name')['type']

#  Calculate total students
total_student = school_name['Student ID'].count()

# Total school budget
total_school_budget = school_data.set_index('school_name')['budget']

# per student budget
budget_per_student = (school_data.set_index('school_name')['budget']/school_data.set_index('school_name')['size'])

# Average Math Score
average_math_score = school_name['math_score'].mean()

# Average Reading Score
average_reading_score = school_name['reading_score'].mean()

# % Passing Math
pass_math_percent = school_data_complete[school_data_complete['math_score'] >= 70].groupby('school_name')['Student ID'].count()/total_student*100

# % Passing Reading
pass_read_percent = school_data_complete[school_data_complete['reading_score'] >= 70].groupby('school_name')['Student ID'].count()/total_student*100

# % Overall Passing (The percentage of students that passed math and reading.
overall_pass = school_data_complete[(school_data_complete['reading_score'] >= 70) & (school_data_complete['math_score'] >= 70)].groupby('school_name')['Student ID'].count()/total_student*100

school_summary = pd.DataFrame({
    "School Type": school_type,
    "Total Students": total_student,
    "Per Student Budget": budget_per_student,
    "Total School Budget": total_school_budget,
    "Average Math Score": average_math_score,
    "Average Reading Score": average_reading_score,
    '% Passing Math': pass_math_percent,
    '% Passing Reading': pass_read_percent,
    "% Overall Passing": overall_pass
})



school_summary = school_summary[['School Type', 
                          'Total Students', 
                          'Total School Budget', 
                          'Per Student Budget', 
                          'Average Math Score', 
                          'Average Reading Score',
                          '% Passing Math',
                          '% Passing Reading',
                          '% Overall Passing']]


# Formatting
school_summary.style.format({'Total Students': '{:}',
                          "Total School Budget": "${:,.2f}",
                          "Per Student Budget": "${:.2f}",
                          'Average Math Score': "{:6f}", 
                          'Average Reading Score': "{:6f}", 
                          "% Passing Math": "{:6f}", 
                          "% Passing Reading": "{:6f}"})

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [4]:
# Sort and display the top five schools by passing rate 
top_perform = school_summary.sort_values("% Overall Passing", ascending = False)
top_perform.head().style.format({'Total Students': '{:}',
                           "Total School Budget": "${:,.2f}", 
                           "Per Student Budget": "${:.2f}", 
                           "% Passing Math": "{:6f}", 
                           "% Passing Reading": "{:6f}", 
                           "% Overall Passing": "{:6f}"})

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [5]:
# Sort and display the bottom five schools by passing rate 
bottom_perform = top_perform.tail()
bottom_perform = bottom_perform.sort_values('% Overall Passing')
bottom_perform.style.format({'Total Students': '{: }', 
                       "Total School Budget": "${:,.2f}", 
                       "Per Student Budget": "${:.2f}", 
                       "% Passing Math": "{:6f}", 
                       "% Passing Reading": "{:6f}", 
                       "% Overall Passing": "{:6f}"})

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [6]:
# Grade level average math scores for each school 
ninth_math = student_data.loc[student_data['grade'] == '9th'].groupby('school_name')["math_score"].mean()
tenth_math = student_data.loc[student_data['grade'] == '10th'].groupby('school_name')["math_score"].mean()
eleventh_math = student_data.loc[student_data['grade'] == '11th'].groupby('school_name')["math_score"].mean()
twelfth_math = student_data.loc[student_data['grade'] == '12th'].groupby('school_name')["math_score"].mean()

math_scores = pd.DataFrame({
        "9th": ninth_math,
        "10th": tenth_math,
        "11th": eleventh_math,
        "12th": twelfth_math
})
math_scores = math_scores[['9th', '10th', '11th', '12th']]
math_scores.index.name = "School Name"

# Show and format
math_scores.style.format({'9th': '{:.6f}', 
                          "10th": '{:.6f}', 
                          "11th": "{:.6f}", 
                          "12th": "{:.6f}"})

,9th,10th,11th,12th
School Name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [7]:
# Grade level average reading scores for each school
ninth_reading = student_data.loc[student_data['grade'] == '9th'].groupby('school_name')["reading_score"].mean()
tenth_reading = student_data.loc[student_data['grade'] == '10th'].groupby('school_name')["reading_score"].mean()
eleventh_reading = student_data.loc[student_data['grade'] == '11th'].groupby('school_name')["reading_score"].mean()
twelfth_reading = student_data.loc[student_data['grade'] == '12th'].groupby('school_name')["reading_score"].mean()

# Merge reading score averages by school and grade together
reading_scores = pd.DataFrame({
        "9th": ninth_reading,
        "10th": tenth_reading,
        "11th": eleventh_reading,
        "12th": twelfth_reading
})
reading_scores = reading_scores[['9th', '10th', '11th', '12th']]
reading_scores.index.name = "School Name"

# Format
reading_scores.style.format({'9th': '{:.6f}', 
                             "10th": '{:.6f}', 
                             "11th": "{:.6f}", 
                             "12th": "{:.6f}"})

,9th,10th,11th,12th
School Name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [8]:
# Create bins
bins = [0, 585, 630, 645, 680]

# Create names for the bins
names = ["<$585", "$585-630", "$630-645", "$645-680"]

# Categorize the spendings 
school_summary["Spending Ranges (Per Student)"] = pd.cut(budget_per_student,bins,labels = names)

# Scores Based on Spending Ranges
avg_math_score_spending = round(school_summary.groupby(["Spending Ranges (Per Student)"]).mean()["Average Math Score"],2)
avg_reading_score_spending = round(school_summary.groupby(["Spending Ranges (Per Student)"]).mean()["Average Reading Score"],2)
students_passing_math_spending = round(school_summary.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Math"],2)
students_passing_reading_spending = round(school_summary.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Reading"],2)
overall_pass_rate_spending = round(school_summary.groupby(["Spending Ranges (Per Student)"]).mean()["% Overall Passing"],2)


scores_based_spending = pd.DataFrame({"Average Math Score":avg_math_score_spending,
                                     "Average Reading Score":avg_reading_score_spending,
                                     "% Passing Math":students_passing_math_spending,
                                     "%Passing Reading":students_passing_reading_spending ,
                                     "% Overall Passing":overall_pass_rate_spending})


# Display dataframe
scores_based_spending

,Average Math Score,Average Reading Score,% Passing Math,%Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$585,83.46,83.93,93.46,96.61,90.37
$585-630,81.90,83.16,87.13,92.72,81.42
$630-645,78.52,81.62,73.48,84.39,62.86
$645-680,77.00,81.03,66.16,81.13,53.53


## Scores by School Size

* Perform the same operations as above, based on school size.

In [9]:
# Create bins
bins2 = [0, 1000, 2000, 5000]

# Create names for bins
size_range = ["Small (<1000)","Medium (1000-2000)","Large (2000-5000)"]


school_summary["School Size"] = pd.cut(school_summary["Total Students"],bins2, labels=size_range)


size_summary = school_summary.groupby("School Size")
size_summary = size_summary[["Average Math Score", "Average Reading Score", "% Passing Math", 
                             "% Passing Reading", "% Overall Passing"]]
size_summary.mean()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,89.883853
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,90.621535
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,58.286003


## Scores by School Type

* Perform the same operations as above, based on school type

In [10]:
type_summary = school_summary.groupby("School Type")
type_summary = type_summary[["Average Math Score", "Average Reading Score", "% Passing Math", 
                             "% Passing Reading", "% Overall Passing"]]
type_summary.mean()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,90.432244
District,76.956733,80.966636,66.548453,80.799062,53.672208


## Analysis

•	The dataset has a total of 15 schools and 39,170 students. The percent of students passing reading (85.80%) and their average reading score of 91.87 is higher than the percent of students passing math (74.98%) and their average math score of 78.98, which reveals that students are doing better in reading than math.

•	The school summary shows that from the 15 schools, 7 district schools have more students than the 8 charter schools which can mean that the district schools’ geographical coverage can be a reason for their larger student population/ size.


•	Based on the performance of spending ranges per student, students have a higher overall passing rate of 90.37 percent in the lower spending ranges (<$585). 


•	Small and medium sized schools performed better than large sized schools on passing math. There's 93.55 percent in small sized schools and 93.59 percent in medium sized schools passing math while there's only 69.96 percent in large sized schools passing math. 

•	The data shows that the Charter Schools are scoring better than the District Schools across all metrics (average, percent, and overall). Pena High School has the average math score (83.839917) with (94.59%) passing math and average reading score (84.044699) with (95.94%) passing reading while Huang High School has the average math score (76.629414) with (65.68%) passing math and average reading score (81.182722) with (81.31%) passing reading. Since Pena High School is a charter school with better scores and smaller student population while Huang High School is a district school with lower scores and larger student population, this can mean that schools with larger student population/ size negatively influences student achievements.
